<a href="https://colab.research.google.com/github/nutworker/qM-AI-L/blob/main/QnA_Llama_3_1_8b_with_Unsloth_D2_trial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
repository_url = 'https://github.com/nutworker/qM-AI-L'
!git clone {repository_url}

Cloning into 'qM-AI-L'...
remote: Enumerating objects: 656, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 656 (delta 166), reused 73 (delta 73), pack-reused 427 (from 1)
Receiving objects: 100% (656/656), 16.05 MiB | 13.99 MiB/s, done.
Resolving deltas: 100% (350/350), done.


In [2]:
# %%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-30jknb5k/unsloth_775d50e90d744313a0b7cad5978ffc43
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-30jknb5k/unsloth_775d50e90d744313a0b7cad5978ffc43
  Resolved https://github.com/unslothai/unsloth.git to commit c3f4e9a87d964ecee1efd9963f497119edbefaab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 211.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00


In [3]:
# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

# **Setting Up the Environment**

In [4]:
!pip install rouge_score
from rouge_score import rouge_scorer

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1536b213b2a40dba59182d2d339665006df8ccd0b19cef73cfc5bc59904255a2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [5]:
# Importing Libraries

import os
import re
import pandas as pd # Data Handling
import torch
import tensorflow as tf
import numpy as np
import time
from google.colab import files
import shutil


In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
!pip install datasets

In [8]:
from torch import __version__
from packaging.version import Version as V
if V(__version__) < V("2.4.0"):
  xformers = "xformers==0.0.27"
else:
  xformers =  "xformers"

In [9]:
import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [11]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True
)


==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [12]:
# tokenizer.eos_token = "<|endoftext|>"
EOS_TOKEN = tokenizer.eos_token
BOS_TOKEN = tokenizer.bos_token

EOS_TOKEN, BOS_TOKEN

('<|end_of_text|>', '<|begin_of_text|>')

In [13]:
# tokenizer.eos_token = "<|endoftext|>"  # Set the EOS token if not done
# tokenizer.bos_token = "<|beginoftext|>"  # Set the EOS token if not done
model.config.eos_token_id = tokenizer.eos_token_id
model.config.bos_token_id = tokenizer.bos_token_id

# **Downloading 4-bit Quantized Model and Adding LoRA Adapters**

# **Dataset-1 :Preparing the Dataset for Fine-tuning**

In [14]:
D2_train_path = '/content/qM-AI-L/QA-System/Dataset-2/train.csv'
D2_test_path = '/content/qM-AI-L/QA-System/Dataset-2/test.csv'
D2_dev_path = '/content/qM-AI-L/QA-System/Dataset-2/dev.csv'


In [15]:
D2_train_df = pd.read_csv(D2_train_path)
D2_test_df = pd.read_csv(D2_test_path)
D2_dev_df = pd.read_csv(D2_dev_path)


In [16]:
D2_train_df.shape, D2_test_df.shape, D2_dev_df.shape

((1985, 2), (249, 2), (248, 2))

In [17]:
D2_train_df.columns,  D2_test_df.columns,  D2_dev_df.columns

(Index(['question', 'answer'], dtype='object'),
 Index(['question', 'answer'], dtype='object'),
 Index(['question', 'answer'], dtype='object'))

## **Add prompt format column to train df for model input**

In [18]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "answer for this question", # instruction
        "Is 9.11 larger than 9.9?", # input
        "", # output—leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 50)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
answer for this question

### Input:
Is 9.11 larger than 9.9?

### Response:
Yes, 9.11 is larger than 9.9.

### Instruction:
answer for this question

### Input:
Is 9.9 larger than 9.11?

### Response:
No, 9.9 is not larger than


In [19]:
tokenizer.eos_token

'<|end_of_text|>'

In [81]:
alpaca_prompt = """Below is an instruction that describes a task.
Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatted_train(x):
    instructions = """Generate an answer for the following question in less than three sentences."""
    # Provide a answer without including any reference links or citations."""
    inputs       = x['question']
    outputs      = x['answer']
    texts = []
    # for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
    text = alpaca_prompt.format(instructions, inputs, outputs) + EOS_TOKEN
        # texts.append(text)
    return text

In [82]:
def format_test(x):
    instructions = """Generate an answer for the following question in less than three sentences."""
    # Provide a answer without including any reference links or citations."""
    inputs       = x['question']
    outputs      = ""
    texts = []
    # for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
    text = alpaca_prompt.format(instructions, inputs, outputs) + EOS_TOKEN
        # texts.append(text)
    return text

In [83]:
# #Function for Test and Dev dataset
# def format_test(x):
#   instruction = "Generate an answer for the following question in less than three sentences."

#   if x['question']:
#     formatted_text = f""" {BOS_TOKEN} Below is an instruction that describes a task. \
#     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

#     ### Instruction:
#     {instruction}

#     ### Input:
#     {x['question']}

#     ### Response:
#      {EOS_TOKEN}"""

#   else:
#     formatted_text = f"""Below is an instruction that describes a task. \
#     Write a response that appropriately completes the request.

#     ### Instruction:
#     {instruction}

#     ### Response:
#      {EOS_TOKEN}"""

#   return formatted_text

In [84]:
D2_train_df

,question,answer,formatted_text
0,What is the role of Siamese networks in domain...,Siamese networks can be used to align the repr...,Below is an instruction that describes a task....
1,What is a recurrent neural network (RNN)?,"An RNN handles sequence data, capturing tempor...",Below is an instruction that describes a task....
2,What are Eigen values and Eigen vectors in the...,Eigen values represent the variance captured b...,Below is an instruction that describes a task....
3,What are Deep Q-Networks?,Deep Q-networks are neural networks that appro...,Below is an instruction that describes a task....
4,What properties of words can Word2Vec extract?,Word2Vec can extract and provide the most simi...,Below is an instruction that describes a task....
...,...,...,...
1980,What is the method of vectorization in Bag-of-...,The simplest method is to mark the presence of...,Below is an instruction that describes a task....
1981,What is the significance of feature interactions?,Feature interactions capture relationships bet...,Below is an instruction that describes a task....
1982,What is the difference between hard margin and...,Hard margin SVM assumes that the data is linea...,Below is an instruction that describes a task....
1983,How does quantization error change with differ...,Quantization error decreases with larger data ...,Below is an instruction that describes a task....


In [85]:
D2_train_df["formatted_text"] = D2_train_df.apply(formatted_train, axis=1)
D2_test_df["formatted_text"] = D2_test_df.apply(format_test, axis=1)
D2_dev_df["formatted_text"] = D2_dev_df.apply(format_test, axis=1)

**Check train prompt includes answer column/ data, and test and dev prompts does not include answers/targets**

In [86]:
D2_train_df

,question,answer,formatted_text
0,What is the role of Siamese networks in domain...,Siamese networks can be used to align the repr...,Below is an instruction that describes a task....
1,What is a recurrent neural network (RNN)?,"An RNN handles sequence data, capturing tempor...",Below is an instruction that describes a task....
2,What are Eigen values and Eigen vectors in the...,Eigen values represent the variance captured b...,Below is an instruction that describes a task....
3,What are Deep Q-Networks?,Deep Q-networks are neural networks that appro...,Below is an instruction that describes a task....
4,What properties of words can Word2Vec extract?,Word2Vec can extract and provide the most simi...,Below is an instruction that describes a task....
...,...,...,...
1980,What is the method of vectorization in Bag-of-...,The simplest method is to mark the presence of...,Below is an instruction that describes a task....
1981,What is the significance of feature interactions?,Feature interactions capture relationships bet...,Below is an instruction that describes a task....
1982,What is the difference between hard margin and...,Hard margin SVM assumes that the data is linea...,Below is an instruction that describes a task....
1983,How does quantization error change with differ...,Quantization error decreases with larger data ...,Below is an instruction that describes a task....


In [87]:
print(D2_train_df['formatted_text'][1])

Below is an instruction that describes a task.
Write a response that appropriately completes the request.

### Instruction:
Generate an answer for the following question in less than three sentences.

### Input:
What is a recurrent neural network (RNN)?

### Response:
An RNN handles sequence data, capturing temporal dependencies with cycles in connections.<|end_of_text|>


In [88]:
D2_test_df['formatted_text'][0]

'Below is an instruction that describes a task.\nWrite a response that appropriately completes the request.\n\n### Instruction:\nGenerate an answer for the following question in less than three sentences.\n\n### Input:\nWhat is feature selection?\n\n### Response:\n<|end_of_text|>'

In [89]:
D2_dev_df['formatted_text'][0]

'Below is an instruction that describes a task.\nWrite a response that appropriately completes the request.\n\n### Instruction:\nGenerate an answer for the following question in less than three sentences.\n\n### Input:\nHow do U-Nets skip connections contribute to better segmentation performance?\n\n### Response:\n<|end_of_text|>'

# **Convert DFs to Datasets**

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [91]:
from datasets import Dataset
D2_Train_dataset = Dataset.from_pandas(D2_train_df)
D2_Test_dataset = Dataset.from_pandas(D2_test_df)
D2_Dev_dataset = Dataset.from_pandas(D2_dev_df)

In [92]:
D2_Train_dataset, D2_Test_dataset, D2_Dev_dataset

(Dataset({
     features: ['question', 'answer', 'formatted_text'],
     num_rows: 1985
 }),
 Dataset({
     features: ['question', 'answer', 'formatted_text'],
     num_rows: 249
 }),
 Dataset({
     features: ['question', 'answer', 'formatted_text'],
     num_rows: 248
 }))

In [93]:
#check train dataset
D2_Train_dataset['formatted_text'][0:2]

['Below is an instruction that describes a task.\nWrite a response that appropriately completes the request.\n\n### Instruction:\nGenerate an answer for the following question in less than three sentences.\n\n### Input:\nWhat is the role of Siamese networks in domain adaptation?\n\n### Response:\nSiamese networks can be used to align the representations of source and target domains by minimizing the discrepancy between them, making the model more robust to domain shifts.<|end_of_text|>',
 'Below is an instruction that describes a task.\nWrite a response that appropriately completes the request.\n\n### Instruction:\nGenerate an answer for the following question in less than three sentences.\n\n### Input:\nWhat is a recurrent neural network (RNN)?\n\n### Response:\nAn RNN handles sequence data, capturing temporal dependencies with cycles in connections.<|end_of_text|>']

# **Fine-tuning Google Gemma for Code Dataset**


In [94]:
from trl import SFTTrainer
from transformers import TrainingArguments

In [95]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = True, # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [96]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = D2_Train_dataset,
    dataset_text_field = "formatted_text",
    max_seq_length = 1024,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 3e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1985 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [97]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,985 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.223900
2,1.311200
3,1.319800
4,1.284800
5,1.273500
6,1.181900
7,1.263700
8,1.192900
9,1.161500
10,1.355100


**#check test data for infrencing and answer generation**

In [98]:
from transformers import TextStreamer

In [99]:
D2_test_df['formatted_text'][2], D2_test_df['formatted_text'][3]

('Below is an instruction that describes a task.\nWrite a response that appropriately completes the request.\n\n### Instruction:\nGenerate an answer for the following question in less than three sentences.\n\n### Input:\nWhat is batch normalization and how does it work?\n\n### Response:\n<|end_of_text|>',
 'Below is an instruction that describes a task.\nWrite a response that appropriately completes the request.\n\n### Instruction:\nGenerate an answer for the following question in less than three sentences.\n\n### Input:\nHow does Word2Vec create word embeddings?\n\n### Response:\n<|end_of_text|>')

# **Checkout the trained model generating answer**

In [100]:
Prompt = D2_test_df['formatted_text'][45]


FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    Prompt
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=50,    eos_token_id=tokenizer.eos_token_id,  # Set EOS token explicitly
    pad_token_id=tokenizer.pad_token_id,  # Ensure padding is handled
    do_sample=True)  # If sampling, or False for greedy search )
# Get the generated text by decoding the IDs
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text


<|begin_of_text|>Below is an instruction that describes a task.
Write a response that appropriately completes the request.

### Instruction:
Generate an answer for the following question in less than three sentences.

### Input:
What are the components of a confusion matrix?

### Response:
<|end_of_text|><|begin_of_text|>php include("footer.php");?<|end_of_text|>


# **Function to extract answer part from the model generated response**

In [101]:
generated_text

'Below is an instruction that describes a task.\nWrite a response that appropriately completes the request.\n\n### Instruction:\nGenerate an answer for the following question in less than three sentences.\n\n### Input:\nWhat are the components of a confusion matrix?\n\n### Response:\nphp include("footer.php");?'

In [102]:

# **Function to extract answer part from the model generated response**
def get_answer(text):
    start_tag = "### Response:"

    # Find the start and end indices
    start_idx = text.find(start_tag)

    # Check if both tags are found
    if start_idx == -1:
        return None  # Tags not found

    # Extract content between the tags
    answer = text[start_idx + len(start_tag):].strip()

    return answer

get_answer(generated_text)

'php include("footer.php");?'

In [103]:
D2_test_df.columns, D2_test_df.shape

(Index(['question', 'answer', 'formatted_text'], dtype='object'), (249, 3))

In [104]:
# prompt: slice D2_test_df to 100 rows randomly

D2_test_df_sample = D2_test_df.sample(n=20, random_state=42)


In [105]:
D2_test_df_sample['formatted_text'][6]

'Below is an instruction that describes a task.\nWrite a response that appropriately completes the request.\n\n### Instruction:\nGenerate an answer for the following question in less than three sentences.\n\n### Input:\nWhat is variance in machine learning?\n\n### Response:\n<|end_of_text|>'

In [106]:
original_answers = []
llama_answers = []

for index, row in D2_test_df_sample.iterrows():
    original_answer = row['answer']


    Prompt = row['formatted_text']

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    model.to(device)
    inputs = tokenizer(
    [
        Prompt
    ], return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=50,     eos_token_id=tokenizer.eos_token_id,  # Set EOS token explicitly
    pad_token_id=tokenizer.pad_token_id,  # Ensure padding is handled
    do_sample=True)  # If sampling, or False for greedy search)
    # Get the generated text by decoding the IDs
    llama_answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text
    llama_answers.append(get_answer(llama_answer))


    original_answers.append(original_answer)

zipped_subjects = list(zip(original_answers, llama_answers))

D2_Test_Llama_with_Unsloth_df = pd.DataFrame(zipped_subjects, columns = ['original_answers', 'Llama_answers'])
D2_Test_Llama_with_Unsloth_df

<|begin_of_text|>Below is an instruction that describes a task.
Write a response that appropriately completes the request.

### Instruction:
Generate an answer for the following question in less than three sentences.

### Input:
How do word embeddings capture semantic relationships between words?

### Response:
<|end_of_text|><|begin_of_text|>* Word embeddings capture semantic relationships between words by encoding words as vectors in a high-dimensional space where similar words are represented by vectors that are close together.<|end_of_text|>
<|begin_of_text|>Below is an instruction that describes a task.
Write a response that appropriately completes the request.

### Instruction:
Generate an answer for the following question in less than three sentences.

### Input:
What is variance in machine learning?

### Response:
<|end_of_text|><|begin_of_text|>php include_once("/home/vh/linh/linh/linh/linh/linh/linh/linh/linh/linh/linh/linh/linh/linh/linh
<|begin_of_text|>Below is an instruct

,original_answers,Llama_answers
0,Words with similar meanings are represented by...,* Word embeddings capture semantic relationshi...
1,Variance refers to the error introduced by the...,"php include_once(""/home/vh/linh/linh/linh/linh..."
2,A cross-validation score is the performance me...,"php include_once(""../../../img/top.gif"")?"
3,PEFT refers to techniques that allow fine-tuni...,:// Parameter-Efficient Fine-Tuning (PEFT) is ...
4,Feature selection helps prevent overfitting by...,:// Feature selection is a process of selectin...
5,A fully connected layer connects each neuron t...,php print_r($answer);?\nA fully connected laye...
6,"GPT-3 is an advanced version of GPT-2, with 17...",:// GPT-3 is a large language model developed ...
7,TF-IDF is the product of TF and IDF.,2nbsp;\nTF-IDF is calculated by multiplying th...
8,Feature engineering for unstructured data invo...,HuggingFace has a number of models for feature...
9,Eigenvalues and eigenvectors are scalar values...,* Eigenvalues and Eigenvectors are the scalar ...


In [107]:
# prompt: Using dataframe D1_Test_Gemma_with_Unsloth_df: get all rows which that are not None in Llama_answers

# Select rows where Llama_answers is not null
D2_Test_Llama3_with_Unsloth = D2_Test_Llama_with_Unsloth_df[D2_Test_Llama_with_Unsloth_df['Llama_answers'].notna()].reset_index()
D2_Test_Llama3_with_Unsloth


,index,original_answers,Llama_answers
0,0,Words with similar meanings are represented by...,* Word embeddings capture semantic relationshi...
1,1,Variance refers to the error introduced by the...,"php include_once(""/home/vh/linh/linh/linh/linh..."
2,2,A cross-validation score is the performance me...,"php include_once(""../../../img/top.gif"")?"
3,3,PEFT refers to techniques that allow fine-tuni...,:// Parameter-Efficient Fine-Tuning (PEFT) is ...
4,4,Feature selection helps prevent overfitting by...,:// Feature selection is a process of selectin...
5,5,A fully connected layer connects each neuron t...,php print_r($answer);?\nA fully connected laye...
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",:// GPT-3 is a large language model developed ...
7,7,TF-IDF is the product of TF and IDF.,2nbsp;\nTF-IDF is calculated by multiplying th...
8,8,Feature engineering for unstructured data invo...,HuggingFace has a number of models for feature...
9,9,Eigenvalues and eigenvectors are scalar values...,* Eigenvalues and Eigenvectors are the scalar ...


# ** Evaluate model performance- Rouge**

In [108]:
!pip install rouge-score  # Install the rouge-score library
from rouge_score import rouge_scorer

In [109]:
D2_test_scores = D2_Test_Llama3_with_Unsloth.copy()

In [110]:
from rouge_score import rouge_scorer

def calculate_rouge(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True) # Instantiate a RougeScorer object
    scores = scorer.score(reference, hypothesis) # Call the score method on the RougeScorer object
    return {metric: score.fmeasure for metric, score in scores.items()}

# Apply ROUGE score calculation to each row
D2_test_scores['rouge_scores'] = D2_test_scores.apply(lambda row: calculate_rouge(row['original_answers'], row['Llama_answers']), axis=1)

In [111]:
D2_test_scores

,index,original_answers,Llama_answers,rouge_scores
0,0,Words with similar meanings are represented by...,* Word embeddings capture semantic relationshi...,"{'rouge1': 0.5238095238095237, 'rouge2': 0.2, ..."
1,1,Variance refers to the error introduced by the...,"php include_once(""/home/vh/linh/linh/linh/linh...","{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ..."
2,2,A cross-validation score is the performance me...,"php include_once(""../../../img/top.gif"")?","{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ..."
3,3,PEFT refers to techniques that allow fine-tuni...,:// Parameter-Efficient Fine-Tuning (PEFT) is ...,"{'rouge1': 0.48148148148148145, 'rouge2': 0.07..."
4,4,Feature selection helps prevent overfitting by...,:// Feature selection is a process of selectin...,"{'rouge1': 0.48275862068965525, 'rouge2': 0.21..."
5,5,A fully connected layer connects each neuron t...,php print_r($answer);?\nA fully connected laye...,"{'rouge1': 0.4680851063829786, 'rouge2': 0.311..."
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",:// GPT-3 is a large language model developed ...,"{'rouge1': 0.34210526315789475, 'rouge2': 0.16..."
7,7,TF-IDF is the product of TF and IDF.,2nbsp;\nTF-IDF is calculated by multiplying th...,"{'rouge1': 0.28571428571428575, 'rouge2': 0.12..."
8,8,Feature engineering for unstructured data invo...,HuggingFace has a number of models for feature...,"{'rouge1': 0.2285714285714286, 'rouge2': 0.060..."
9,9,Eigenvalues and eigenvectors are scalar values...,* Eigenvalues and Eigenvectors are the scalar ...,"{'rouge1': 0.6857142857142857, 'rouge2': 0.363..."


In [112]:
pd.json_normalize(D2_test_scores['rouge_scores'])

,rouge1,rouge2,rougeL,rougeLsum
0,0.523810,0.200000,0.380952,0.380952
1,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000
3,0.481481,0.076923,0.333333,0.333333
4,0.482759,0.214286,0.413793,0.413793
5,0.468085,0.311111,0.425532,0.425532
6,0.342105,0.162162,0.210526,0.210526
7,0.285714,0.121212,0.285714,0.285714
8,0.228571,0.060606,0.114286,0.114286
9,0.685714,0.363636,0.571429,0.571429


In [113]:
D2_test_scores = pd.concat([D2_test_scores, pd.json_normalize(D2_test_scores['rouge_scores'])], axis=1)


In [114]:
D2_test_scores

,index,original_answers,Llama_answers,rouge_scores,rouge1,rouge2,rougeL,rougeLsum
0,0,Words with similar meanings are represented by...,* Word embeddings capture semantic relationshi...,"{'rouge1': 0.5238095238095237, 'rouge2': 0.2, ...",0.523810,0.200000,0.380952,0.380952
1,1,Variance refers to the error introduced by the...,"php include_once(""/home/vh/linh/linh/linh/linh...","{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.000000,0.000000,0.000000,0.000000
2,2,A cross-validation score is the performance me...,"php include_once(""../../../img/top.gif"")?","{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.000000,0.000000,0.000000,0.000000
3,3,PEFT refers to techniques that allow fine-tuni...,:// Parameter-Efficient Fine-Tuning (PEFT) is ...,"{'rouge1': 0.48148148148148145, 'rouge2': 0.07...",0.481481,0.076923,0.333333,0.333333
4,4,Feature selection helps prevent overfitting by...,:// Feature selection is a process of selectin...,"{'rouge1': 0.48275862068965525, 'rouge2': 0.21...",0.482759,0.214286,0.413793,0.413793
5,5,A fully connected layer connects each neuron t...,php print_r($answer);?\nA fully connected laye...,"{'rouge1': 0.4680851063829786, 'rouge2': 0.311...",0.468085,0.311111,0.425532,0.425532
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",:// GPT-3 is a large language model developed ...,"{'rouge1': 0.34210526315789475, 'rouge2': 0.16...",0.342105,0.162162,0.210526,0.210526
7,7,TF-IDF is the product of TF and IDF.,2nbsp;\nTF-IDF is calculated by multiplying th...,"{'rouge1': 0.28571428571428575, 'rouge2': 0.12...",0.285714,0.121212,0.285714,0.285714
8,8,Feature engineering for unstructured data invo...,HuggingFace has a number of models for feature...,"{'rouge1': 0.2285714285714286, 'rouge2': 0.060...",0.228571,0.060606,0.114286,0.114286
9,9,Eigenvalues and eigenvectors are scalar values...,* Eigenvalues and Eigenvectors are the scalar ...,"{'rouge1': 0.6857142857142857, 'rouge2': 0.363...",0.685714,0.363636,0.571429,0.571429


In [115]:
# prompt: drop rouge_scores column

D2_test_scores = D2_test_scores.drop('rouge_scores', axis=1)
D2_test_scores

,index,original_answers,Llama_answers,rouge1,rouge2,rougeL,rougeLsum
0,0,Words with similar meanings are represented by...,* Word embeddings capture semantic relationshi...,0.523810,0.200000,0.380952,0.380952
1,1,Variance refers to the error introduced by the...,"php include_once(""/home/vh/linh/linh/linh/linh...",0.000000,0.000000,0.000000,0.000000
2,2,A cross-validation score is the performance me...,"php include_once(""../../../img/top.gif"")?",0.000000,0.000000,0.000000,0.000000
3,3,PEFT refers to techniques that allow fine-tuni...,:// Parameter-Efficient Fine-Tuning (PEFT) is ...,0.481481,0.076923,0.333333,0.333333
4,4,Feature selection helps prevent overfitting by...,:// Feature selection is a process of selectin...,0.482759,0.214286,0.413793,0.413793
5,5,A fully connected layer connects each neuron t...,php print_r($answer);?\nA fully connected laye...,0.468085,0.311111,0.425532,0.425532
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",:// GPT-3 is a large language model developed ...,0.342105,0.162162,0.210526,0.210526
7,7,TF-IDF is the product of TF and IDF.,2nbsp;\nTF-IDF is calculated by multiplying th...,0.285714,0.121212,0.285714,0.285714
8,8,Feature engineering for unstructured data invo...,HuggingFace has a number of models for feature...,0.228571,0.060606,0.114286,0.114286
9,9,Eigenvalues and eigenvectors are scalar values...,* Eigenvalues and Eigenvectors are the scalar ...,0.685714,0.363636,0.571429,0.571429


In [116]:
!pip install nltk sacrebleu

In [117]:
import sacrebleu
from nltk.translate import meteor_score
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [118]:
# Tokenization function
def tokenize(text):
    return text.lower().split()


In [119]:
def calculate_bleu(reference, hypothesis):
    # BLEU requires tokenized input for reference, but not for hypothesis
    reference = [tokenize(reference)]
    bleu = sacrebleu.corpus_bleu([hypothesis], reference) # hypothesis should be a string
    return bleu.score


D2_test_scores['bleu_score'] = D2_test_scores.apply(lambda row: calculate_bleu(row['original_answers'], row['Llama_answers']), axis=1)

# Calculate METEOR score
def calculate_meteor(reference, hypothesis):
    # Tokenize both reference and hypothesis
    tokenized_reference = tokenize(reference)
    tokenized_hypothesis = tokenize(hypothesis)
    return meteor_score.single_meteor_score(tokenized_reference, tokenized_hypothesis)  # Pass tokenized inputs

D2_test_scores['meteor_score'] = D2_test_scores.apply(lambda row: calculate_meteor(row['original_answers'], row['Llama_answers']), axis=1)

In [120]:
D2_test_scores

,index,original_answers,Llama_answers,rouge1,rouge2,rougeL,rougeLsum,bleu_score,meteor_score
0,0,Words with similar meanings are represented by...,* Word embeddings capture semantic relationshi...,0.523810,0.200000,0.380952,0.380952,1.286713,0.446930
1,1,Variance refers to the error introduced by the...,"php include_once(""/home/vh/linh/linh/linh/linh...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,A cross-validation score is the performance me...,"php include_once(""../../../img/top.gif"")?",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,PEFT refers to techniques that allow fine-tuni...,:// Parameter-Efficient Fine-Tuning (PEFT) is ...,0.481481,0.076923,0.333333,0.333333,0.000000,0.218708
4,4,Feature selection helps prevent overfitting by...,:// Feature selection is a process of selectin...,0.482759,0.214286,0.413793,0.413793,0.000000,0.409515
5,5,A fully connected layer connects each neuron t...,php print_r($answer);?\nA fully connected laye...,0.468085,0.311111,0.425532,0.425532,0.918708,0.527707
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",:// GPT-3 is a large language model developed ...,0.342105,0.162162,0.210526,0.210526,0.000000,0.135542
7,7,TF-IDF is the product of TF and IDF.,2nbsp;\nTF-IDF is calculated by multiplying th...,0.285714,0.121212,0.285714,0.285714,0.000000,0.325387
8,8,Feature engineering for unstructured data invo...,HuggingFace has a number of models for feature...,0.228571,0.060606,0.114286,0.114286,2.627962,0.047393
9,9,Eigenvalues and eigenvectors are scalar values...,* Eigenvalues and Eigenvectors are the scalar ...,0.685714,0.363636,0.571429,0.571429,0.000000,0.532369


In [121]:
average_scores = {
    'average_rouge1': D2_test_scores['rouge1'].mean(),
    'average_rouge2': D2_test_scores['rouge2'].mean(),
    'average_rougeL': D2_test_scores['rougeL'].mean(),
    'average_rougeLsum': D2_test_scores['rougeLsum'].mean(),
    'average_bleu_score': D2_test_scores['bleu_score'].mean(),
    'average_meteor_score': D2_test_scores['meteor_score'].mean(),
}

In [122]:
print("Average Scores for Dataset-2 (Test-Answer-1) QnA task using LLaMA 8B_with_Unsloth model:\n")
for metric, score in average_scores.items():
    print(f"{metric}: {score:.4f}")

Average Scores for Dataset-2 (Test-Answer-1) QnA task using LLaMA 8B_with_Unsloth model:

average_rouge1: 0.3096
average_rouge2: 0.1216
average_rougeL: 0.2341
average_rougeLsum: 0.2368
average_bleu_score: 0.4413
average_meteor_score: 0.2342


# **Repeat above evaluation process for the 2nd Answer annotation**

In [123]:
original_answers = []
llama_answers = []

for index, row in D2_test_df.iterrows():
    original_answer = row['answer2']


    Prompt = row['formatted_text']

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    model.to(device)
    inputs = tokenizer(
    [
        Prompt
    ], return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)
    # Get the generated text by decoding the IDs
    llama_answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text
    llama_answers.append(get_answer(llama_answer))


    original_answers.append(original_answer)

zipped_subjects = list(zip(original_answers, llama_answers))

D1_Test_llama_with_Unsloth_df_A2 = pd.DataFrame(zipped_subjects, columns = ['answer-2', 'Llama_answers'])
D1_Test_llama_with_Unsloth_df_A2

KeyError: 'answer2'

In [ ]:
D1_test_scores_A2 = D1_Test_llama_with_Unsloth_df_A2.copy()

In [ ]:
D1_test_scores_A2['rouge_scores'] = D1_test_scores_A2.apply(lambda row: calculate_rouge(row['answer-2'], row['Llama_answers']), axis=1)

In [ ]:
D1_test_scores_A2 = pd.concat([D1_test_scores_A2, pd.json_normalize(D1_test_scores_A2['rouge_scores'])], axis=1)

In [ ]:
D1_test_scores_A2

In [ ]:
D1_test_scores_A2['bleu_score'] = D1_test_scores_A2.apply(lambda row: calculate_bleu(row['answer-2'], row['Llama_answers']), axis=1)


D1_test_scores_A2['meteor_score'] = D1_test_scores_A2.apply(lambda row: calculate_meteor(row['answer-2'], row['Llama_answers']), axis=1)

In [ ]:
D1_test_scores_A2

In [ ]:
A2_average_scores = {
    'average_rouge1': D1_test_scores_A2['rouge1'].mean(),
    'average_rouge2': D1_test_scores_A2['rouge2'].mean(),
    'average_rougeL': D1_test_scores_A2['rougeL'].mean(),
    'average_rougeLsum': D1_test_scores_A2['rougeLsum'].mean(),
    'average_bleu_score': D1_test_scores_A2['bleu_score'].mean(),
    'average_meteor_score': D1_test_scores_A2['meteor_score'].mean(),
}

In [ ]:
print("Average Scores for Dataset-1 (Test-Answer-2) QnA task using Gemma_7b_with_Unsloth model:\n")
for metric, score in A2_average_scores.items():
    print(f"{metric}: {score:.4f}")


In [ ]:
# make a df with A2_average_scores and average_scores

# Create a DataFrame from the dictionaries
D1_Test_Scores = pd.DataFrame([average_scores, A2_average_scores], index=['Llama3-8b_model Vs.Answer-1', 'Llama3-8b_model Vs.Answer-2'])

# Transpose the DataFrame for better readability
D1_Test_Scores = D1_Test_Scores.T

D1_Test_Scores
